In [46]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import requests
from config import gkey
import math
data = pd.read_csv("aac_intakes_outcomes_address_income.csv")

In [47]:
def getPostalCode(geo_data):
    try:
        return geo_data["results"][0]["geometry"]["location"]
    except:
        return None

In [48]:
def getGeoData(zipcode):
    try:
        target_url = "https://maps.googleapis.com/maps/api/geocode/json?" \
            "address=%s&key=%s" % (zipcode+",+Austin,+TX&", gkey)
        response = requests.get(target_url)
        if response.status_code == 200:
            return getPostalCode(response.json())
        else:
            return None
    except:
        return None

In [49]:
print(getGeoData("78733"))

{'lat': 30.33151879999999, 'lng': -97.86671820000001}


In [50]:
zipcodes = pd.DataFrame(data["zipcode"].unique(), columns=["zipcode"])
zipcodes["Lat"] = ""
zipcodes["Lon"] = ""

In [51]:
print(str(len(zipcodes)))
zipcodes.dropna()
print(zipcodes.head())
print(str(len(zipcodes)))

600
  zipcode Lat Lon
0   78758        
1   78757        
2     NaN        
3   78759        
4   78751        
600


In [71]:
for index, row in zipcodes.iterrows():
    if row["zipcode"] != "" and not pd.isnull(row["zipcode"]):
        geo = getGeoData(row["zipcode"])
        zipcodes.loc[index, "Lat"] = geo["lat"]
        zipcodes.loc[index, "Lon"] = geo["lng"]
        print(index, geo)
    else:
        zipcodes.loc[index, "Lat"] = None
        zipcodes.loc[index, "Lon"] = None
        print(index, None)
print("Done")

0 {'lat': 30.3870686, 'lng': -97.70365609999999}
1 {'lat': 30.3568213, 'lng': -97.730807}
2 None
3 {'lat': 30.401356, 'lng': -97.7525352}
4 {'lat': 30.3055711, 'lng': -97.725376}
5 {'lat': 30.1859258, 'lng': -97.90482}
6 {'lat': 30.2175453, 'lng': -97.8558357}
7 {'lat': 30.196311, 'lng': -97.730807}
8 {'lat': 30.2603535, 'lng': -97.7145152}
9 {'lat': 30.2573497, 'lng': -97.86127669999999}
10 {'lat': 30.4570672, 'lng': -97.68194299999999}
11 {'lat': 30.1471783, 'lng': -97.5897372}
12 {'lat': 30.3081307, 'lng': -97.68194299999999}
13 {'lat': 30.432242, 'lng': -97.7145152}
14 {'lat': 30.3889868, 'lng': -97.6710889}
15 {'lat': 30.267153, 'lng': -97.7430608}
16 {'lat': 30.3317278, 'lng': -97.70515800000001}
17 {'lat': 30.267153, 'lng': -97.7430608}
18 {'lat': 30.2920434, 'lng': -97.7118003}
19 {'lat': 30.24567279999999, 'lng': -97.76883579999999}
20 {'lat': 30.4270699, 'lng': -97.8014482}
21 {'lat': 30.33151879999999, 'lng': -97.86671820000001}
22 {'lat': 30.3431045, 'lng': -97.3732655}
23 

In [73]:
zipcodes.to_csv("Address_Latitude.csv", index=False)

In [74]:
zipcodes = pd.read_csv("Address_Latitude.csv")
zipcodes.head()

,zipcode,Lat,Lon
0,78758,30.387069,-97.703656
1,78757,30.356821,-97.730807
2,NaN,NaN,NaN
3,78759,30.401356,-97.752535
4,78751,30.305571,-97.725376


In [75]:
data_address = pd.merge(data, zipcodes, how="left", on="zipcode")

In [76]:
data_address.head()

,age_upon_outcome,animal_id_outcome,date_of_birth,outcome_subtype,outcome_type,sex_upon_outcome,age_upon_outcome_(days),age_upon_outcome_(years),age_upon_outcome_age_group,outcome_datetime,...,time_in_shelter,time_in_shelter_days,age_upon_intake_number,age_upon_intake_lentime,found_location_street,zipcode,income,population,Lat,Lon
0,10 years,A006100,2007-07-09 00:00:00,NaN,Return to Owner,Neutered Male,3650,10.0,"(7.5, 10.0]",2017-12-07 14:07:00,...,0 days 14:07:00.000000000,0.588194,10,years,Colony Creek And Hunters Trace,78758,"47,244","44,072",30.387069,-97.703656
1,7 years,A006100,2007-07-09 00:00:00,NaN,Return to Owner,Neutered Male,2555,7.0,"(5.0, 7.5]",2014-12-20 16:35:00,...,1 days 06:14:00.000000000,1.259722,7,years,8700 Research Blvd,78757,"62,992","21,310",30.356821,-97.730807
2,6 years,A006100,2007-07-09 00:00:00,NaN,Return to Owner,Neutered Male,2190,6.0,"(5.0, 7.5]",2014-03-08 17:10:00,...,1 days 02:44:00.000000000,1.113889,6,years,8700 Research,78757,"62,992","21,310",30.356821,-97.730807
3,10 years,A047759,2004-04-02 00:00:00,Partner,Transfer,Neutered Male,3650,10.0,"(7.5, 10.0]",2014-04-07 15:12:00,...,4 days 23:17:00.000000000,4.970139,10,years,NaN,NaN,NaN,NaN,NaN,NaN
4,16 years,A134067,1997-10-16 00:00:00,NaN,Return to Owner,Neutered Male,5840,16.0,"(15.0, 17.5]",2013-11-16 11:54:00,...,0 days 02:52:00.000000000,0.119444,16,years,12034 Research Blvd,78759,"74,379","38,891",30.401356,-97.752535


In [79]:
data_address.to_csv("aac_intakes_outcomes_address_income.csv", index=False)